### T&uuml;rme von Hanoi, Callback Variante  

Der Controller sollte sich nur um das &Uuml;bermitteln eines Befehls an die Game-Komponente k&uuml;mmern m&uuml;ssen, nicht auch noch um die Darstellung. Zu diesem Zweck kommuniziert die
Game-Komponente via Callbacks mit der View:

- Je nach Tastatureingabe ruft der Controller die Funktion `new_game()` oder `move_disk(i, j)` der Game-Komponente auf.  
- Nach getaner Arbeit rufen dann `new_game` bez. `move_disk` das von der View registrierte Callback mit den Argumenten `event` und `data` auf, was zur Darstellung des Spielzustandes f&uuml;hrt.

Die Kommunikation l&auml;uft aber auch noch &uuml;ber die globalen Variablen `stacks`, `callbacks`, `canvas` und `src`.

In [ ]:
# Game
stacks = [[], [], []]   
callbacks = []

def new_game():
    # Liste stacks modifizieren
    stacks[:] = [list(range(4)), [], []]
    _notify('new_game', stacks)
    
def move_disk(i, j):    
    x = stacks[i].pop()
    stacks[j].append(x)
    _notify('move_disk', stacks)
        
def _notify(event, data):
    for f in callbacks:
        f(event, data)

In [ ]:
# View
canvas_width = 300
canvas_height = 100
positions = [50, 150, 250]
disk_widths = [90, 70, 50, 30]
disk_height = 10
stack_width = 100
layout = {'border': '1px solid black'}
colors = ['#8f5902', '#204a87', '#c4a000', '#f57900'] 

from ipycanvas import Canvas
canvas = Canvas(width = canvas_width, height = canvas_height, layout=layout)

def draw_stack(stack, x):
    canvas.clear_rect(x - stack_width/2, 0, stack_width, canvas_height)
    for i, disk in enumerate(stack):
        canvas.fill_style = colors[disk]
        w = disk_widths[disk]
        h = disk_height
        canvas.fill_rect(x - w/2, canvas_height-(i+1)*h, w, h)
        
def draw_stacks(stacks):
    for x, stack in zip(positions, stacks):
        draw_stack(stack, x)

def callback(event, data):
    draw_stacks(data)

callbacks.append(callback)   

In [ ]:
# Controller
src = None

def on_key_down(key, *flags):
    global src
  
    if key in '123' and src is None:
        src = int(key) - 1
    elif key in '123' and src is not None:
        dst = int(key) - 1
        move_disk(src, dst)
        src = None
    elif key == 'n':
        new_game()
        src = None    
    else:
        src = None        

In [ ]:
canvas.on_key_down(on_key_down)
canvas